In [3]:
import itertools
import operator
import pickle
import re
from abc import *
from copy import deepcopy
from operator import itemgetter
from typing import *
import numpy as np
import scipy as sp
import networkx as nx
import mne
import time
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors._dist_metrics import DistanceMetric
from sklearn.utils import shuffle
from typing import List, Tuple
from sklearn.model_selection import train_test_split

from nodestimation.learning.connectoming import make_connectome
from nodestimation.learning.estimation import collect_statistic, \
    compute_importance, collect_cross_statistic, make_selection_map, \
    select, separate_datasets, selected_statistic, choose_best, selected_data, make_feature_selection
from nodestimation.learning.informativeness import CrossInformativeness, Informativeness, SubjectsInformativeness, \
    NodesInformativeness
from nodestimation.learning.networking import sparse_graph, graph_to_hemispheres, hemispheres_division_modularity, \
    metric_for_hemispheres
from nodestimation.processing.features import prepare_features
from nodestimation.project import find_subject_dir, conditions_unique_code
from nodestimation.pipeline import pipeline
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import nibabel
import nilearn.plotting as nplt
from nodestimation.project.actions import read
import nodestimation as nd
from nodestimation.learning.modification import append_series, promote
import nodestimation.learning.modification as lmd
from nodestimation.project.subject import Subject
from sklearn.preprocessing import *
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.feature_selection import SelectFromModel
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPClassifier
import matplotlib as mpl
from nodestimation.learning.selection import SubjectsStatistic, Wilcoxon, Mannwhitneyu, Test
from scipy.stats import wilcoxon
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from scipy.fftpack import fftfreq, irfft, rfft
from scipy.fftpack import fftfreq, irfft, rfft

ENGEL1 = [
    'B1C2',
    'B1R1',
    'G1R1',
    'G1V2',
    'J1T2',
    'K1V1',
    'L1P1',
    'M1G2',
    'M1N2',
    'O1O2',
    'R1D2',
    'S1A2',
    'S1B1',
    'S1H1',
    'S1U3'
]
ENGEL2 = [
    'L2M1',
    'M2S2',
    'N2K2',
    'P1H2'
]
ENGEL3 = [
    'N3S2',
    'S3R1'
]
ENGEL4 = [
    'K4L2'
]
REJECTED = [
    'S1U3',
    'P1H2'
]

AGE = {
    'B1C2': 28.0,
    'B1R1': 16.0,
    'G1R1': 23.0,
    'G1V2': 5.0,
    'J1T2': 8.0,
    'K1V1': 10.0,
    'K4L2': 14.0,
    'L1P1': 16.0,
    'L2M1': 20.0,
    'M1G2': 8.0,
    'M1N2': 7.0,
    'M2S2': 18.0,
    'N2K2': 30.0,
    'N3S2': 10.0,
    'O1O2': 18.0,
    'R1D2': 6.5,
    'P1H2': 7.0,
    'S1A2': 12.0,
    'S1B1': 17.0,
    'S1H1': 28.0,
    'S3R1': 19.0,
    'S1U3': 15.0,
}

SEX = {
    'B1C2': 'f',
    'B1R1': 'm',
    'G1R1': 'f',
    'G1V2': 'm',
    'J1T2': 'f',
    'K1V1': 'f',
    'K4L2': 'f',
    'L1P1': 'f',
    'L2M1': 'f',
    'M1G2': 'm',
    'M1N2': 'm',
    'M2S2': 'm',
    'N2K2': 'm',
    'N3S2': 'm',
    'O1O2': 'f',
    'R1D2': 'f',
    'P1H2': 'm',
    'S1A2': 'm',
    'S1B1': 'm',
    'S1H1': 'm',
    'S3R1': 'm',
    'S1U3': 'f',
}

CONNECTOMES_KIND = 'initial'

SUBJECTS = pipeline(
    methods=['wpli', 'envelope', 'coh', 'imcoh', 'plv', 'ciplv', 'ppc', 'pli', 'pli2_unbiased', 'wpli2_debiased'],
    freq_bands=[(4, 8), (8, 14), (6, 8), (8, 10)],
)

/home/user/.local/lib/python3.9/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


All computation has been already done, loading of the existing file with the solution...


In [4]:
subjects = SUBJECTS.copy()

DATASETS = {
    subject.name: {
        freq: {
            method: make_connectome(subject, freq, method, CONNECTOMES_KIND, threshold=1)
           for method in subject.connectomes[freq]
        }
        for freq in subject.connectomes
    }
    for subject in subjects
}
print('Connectomes are prepared')

Connectomes are prepared


In [5]:
print('Processing centrality...')

datasets = DATASETS.copy()

ignored_methods = [
#     'wpli',
#     'envelope',
#     'coh',
#     'imcoh',
#     'plv',
#     'ciplv',
#     'ppc',
#     'pli',
#     'pli2_unbiased',
#     'wpli2_debiased'
]

close = dict()
between = dict()
eigen = dict()
degree = dict()
info = dict()
harmony = dict()
katz = dict()
percolation = dict()

total_ignored_subjects = 0

for subject_name in datasets:
    is_corrupted = False
    close.update({subject_name: dict()})
    between.update({subject_name: dict()})
    eigen.update({subject_name: dict()})
    degree.update({subject_name: dict()})
    info.update({subject_name: dict()})
    harmony.update({subject_name: dict()})
    katz.update({subject_name: dict()})
    percolation.update({subject_name: dict()})

    for freq in datasets[subject_name]:

        if is_corrupted:
            break

        for method in datasets[subject_name][freq]:

            if method in ignored_methods:
                continue

            print(subject_name, freq, method)
            label_names = datasets[subject_name][freq][method].columns

            if nx.is_connected(
                    nx.convert_matrix.from_numpy_array(
                        datasets[subject_name][freq][method].to_numpy()
                    )
            ):
                arr  = datasets[subject_name][freq][method].to_numpy()
            else:
                try:
                    corrupted_subject = [subject for subject in subjects if subject.name == subject_name][0]
                    subjects.remove(corrupted_subject)
                except IndexError:
                    pass
                total_ignored_subjects += 1
                print(f'Graph is not connected: {subject_name}, {freq}, {method}.\n'
                      f'This subject will be ignored.\n'
                      f'Total of ignored subjects: {total_ignored_subjects}\n'
                      f'Total of spared subjects: {len(subjects)}\n')
                is_corrupted = True
                break
                # raise ValueError(f'Graph is not connected: {subject_name}, {freq}, {method}')
            # if matrix has negative values, shift it to make minimal value at least 0
            if arr.min().min() < 0:
                print(f'\tShifting connectome for {method} at {freq}')
                for i in range(arr.shape[0]):
                    for j in range(arr.shape[1]):
                        if i != j: arr[i, j] -= arr.min().min()
            G = nx.convert_matrix.from_numpy_array(arr)
            mapping = {node: label_name for node, label_name in zip(G, label_names)}
            max = np.max(
                np.array([
                    np.sum(row)
                    for row in arr
                ])
            )
            min = np.min(
                np.array([
                    np.sum(row)
                    for row in arr
                ])
            )
            states = {node: (np.sum(row) - min)/(max - min) for node, row in zip(label_names, arr)}
            G_num = G.copy()
            G = nx.relabel_nodes(G, mapping)
            if CONNECTOMES_KIND == 'initial' or CONNECTOMES_KIND == 'inverse' or CONNECTOMES_KIND == 'spatial':
                G_sup = nx.convert_matrix.from_numpy_array(
                        lmd.suppress(
                            datasets[subject_name][freq][method],
                            trigger= arr.mean().mean(),
                            optimal=0
                        ).to_numpy()
                    )
                G_sup = nx.relabel_nodes(G_sup, mapping)
            else:
                G_sup = G
            for place, data in zip(
                [
                    close[subject_name],
                    between[subject_name],
                    eigen[subject_name],
                    info[subject_name],
                    degree[subject_name],
                    harmony[subject_name],
                    katz[subject_name],
                    percolation[subject_name]
                ],[
                        nx.closeness_centrality(G, distance='weight'),
                        nx.betweenness_centrality(G_sup, weight='weight'),
                        nx.eigenvector_centrality_numpy(G, weight='weight'),
                        nx.information_centrality(G, weight='weight'),
                        dict(G.degree(weight='weight')),
                        nx.harmonic_centrality(G, distance='weight'),
                        nx.katz_centrality_numpy(G, weight='weight'),
                        nx.algorithms.centrality.percolation_centrality(G_sup, weight='weight', states=states)
                    ]
            ):
                place.update({
                    freq + '_' + method: pd.Series(data)
                })

print('Centrality computations are done')

Processing centrality...
M1G2 4-8Hz wpli
M1G2 4-8Hz envelope
M1G2 4-8Hz coh
M1G2 4-8Hz imcoh
	Shifting connectome for imcoh at 4-8Hz
M1G2 4-8Hz plv
M1G2 4-8Hz ciplv
M1G2 4-8Hz ppc
	Shifting connectome for ppc at 4-8Hz
M1G2 4-8Hz pli
M1G2 4-8Hz pli2_unbiased
	Shifting connectome for pli2_unbiased at 4-8Hz
M1G2 4-8Hz wpli2_debiased
	Shifting connectome for wpli2_debiased at 4-8Hz
M1G2 8-14Hz wpli
M1G2 8-14Hz envelope
M1G2 8-14Hz coh
M1G2 8-14Hz imcoh
	Shifting connectome for imcoh at 8-14Hz
M1G2 8-14Hz plv
M1G2 8-14Hz ciplv
M1G2 8-14Hz ppc
M1G2 8-14Hz pli
M1G2 8-14Hz pli2_unbiased
	Shifting connectome for pli2_unbiased at 8-14Hz
M1G2 8-14Hz wpli2_debiased
	Shifting connectome for wpli2_debiased at 8-14Hz
M1G2 6-8Hz wpli
M1G2 6-8Hz envelope
M1G2 6-8Hz coh
M1G2 6-8Hz imcoh
	Shifting connectome for imcoh at 6-8Hz
M1G2 6-8Hz plv
M1G2 6-8Hz ciplv
M1G2 6-8Hz ppc
	Shifting connectome for ppc at 6-8Hz
M1G2 6-8Hz pli
M1G2 6-8Hz pli2_unbiased
	Shifting connectome for pli2_unbiased at 6-8Hz
M1G2 6-

In [20]:
print('Updating connectomes...')

datasets_centrality = dict()

for subject in subjects:

    datasets_centrality.update({subject.name: dict()})
    datasets_centrality[subject.name].update({
        'close': pd.DataFrame(close[subject.name]),
        'between': pd.DataFrame(between[subject.name]),
        'eigen': pd.DataFrame(eigen[subject.name]),
        'info': pd.DataFrame(info[subject.name]),
        'degree': pd.DataFrame(degree[subject.name]),
        'harmony': pd.DataFrame(harmony[subject.name]),
        'katz': pd.DataFrame(katz[subject.name]),
        'percolation': pd.DataFrame(percolation[subject.name])
    })
    true = np.array([node.type == 'resected' for node in subject.nodes])
    datasets_centrality[subject.name]['close'] =\
        datasets_centrality[subject.name]['close'].assign(resected=true)
    datasets_centrality[subject.name]['between'] =\
        datasets_centrality[subject.name]['between'].assign(resected=true)
    datasets_centrality[subject.name]['eigen'] =\
        datasets_centrality[subject.name]['eigen'].assign(resected=true)
    datasets_centrality[subject.name]['degree'] =\
        datasets_centrality[subject.name]['degree'].assign(resected=true)
    datasets_centrality[subject.name]['info'] =\
        datasets_centrality[subject.name]['info'].assign(resected=true)
    datasets_centrality[subject.name]['harmony'] =\
        datasets_centrality[subject.name]['harmony'].assign(resected=true)
    datasets_centrality[subject.name]['katz'] =\
        datasets_centrality[subject.name]['katz'].assign(resected=true)
    datasets_centrality[subject.name]['percolation'] =\
        datasets_centrality[subject.name]['percolation'].assign(resected=true)

Updating connectomes...


In [21]:
print('Saving result')

path = f'/home/user/Documents/NodesEstimation/theta&late_theta&early_alpha&alpha/all_centrality_all_subjects_theta&l-theta&e-alpha&alpha.pkl'
pickle.dump(
    datasets_centrality,
    open(
        path,
        'wb'
    )
)

print(f'Successfully saved at {path}')

Saving result
Successfully saved at /home/user/Documents/NodesEstimation/theta&late_theta&early_alpha&alpha/all_centrality_all_subjects_theta&l-theta&e-alpha&alpha.pkl


In [22]:
print(len(list(datasets_centrality.keys())), len(subjects))

22 22


In [23]:
for subject in subjects:
    subject.datasets = datasets_centrality[subject.name]

print(f'Connectomes successfully updated. Now each subject has new {CONNECTOMES_KIND} connectomes')

print('Collecting statistics...')

stat1 = SubjectsStatistic(subjects, 'resected', centrality_metric='eigen')
print('Eigencentrality statistics are collected')
stat2 = SubjectsStatistic(subjects, 'resected', centrality_metric='between')
print('Betweenness centrality statistics are collected')
stat3 = SubjectsStatistic(subjects, 'resected', centrality_metric='close')
print('Closeness centrality statistics are collected')
stat4 = SubjectsStatistic(subjects, 'resected', centrality_metric='degree')
print('Degree centrality statistics are collected')
stat5 = SubjectsStatistic(subjects, 'resected', centrality_metric='info')
print('Info centrality statistics are collected')
stat6 = SubjectsStatistic(subjects, 'resected', centrality_metric='harmony')
print('Harmony centrality statistics are collected')
stat7 = SubjectsStatistic(subjects, 'resected', centrality_metric='katz')
print('Katz centrality statistics are collected')
stat8 = SubjectsStatistic(subjects, 'resected', centrality_metric='percolation')
print('Percolation centrality statistics are collected')
print(f'All statistics for {CONNECTOMES_KIND} connectomes are collected')
print('Saving statistics')

path = f'/home/user/Documents/NodesEstimation/theta&late_theta&early_alpha&alpha/all_stats_all_subjects_theta&l-theta&e-alpha&alpha.pkl'
pickle.dump(
    (
        stat1,
        stat2,
        stat3,
        stat4,
        stat5,
        stat6,
        stat7,
        stat8,
    ),
    open(
        path,
        'wb'
    )
)

print(f'All statistics are successfully saved at {path = }')


Connectomes successfully updated. Now each subject has new initial connectomes
Eigencentrality statistics are collected
Betweenness centrality statistics are collected
Closeness centrality statistics are collected
Degree centrality statistics are collected
Info centrality statistics are collected
Harmony centrality statistics are collected
Katz centrality statistics are collected
Percolation centrality statistics are collected
All statistics for initial connectomes are collected
Saving statistics
All statistics are successfully saved at path = '/home/user/Documents/NodesEstimation/theta&late_theta&early_alpha&alpha/all_stats_all_subjects_theta&l-theta&e-alpha&alpha.pkl'
